# Steps for Installing

In [1]:
import sys
!{sys.executable} -m pip install pulp
import pulp
from pulp import *
import numpy as np
import pandas as pd
import itertools
from numpy.linalg import matrix_rank
import numpy.linalg
import time
import scipy as sp
from scipy import linalg

# Data Set Up

In [2]:
# List of constants (value can change)
p = 4   # size of P
n = 1   # size of N
l = 3   # size of limit for chains
K = 3   # max number of transplant carried

# Sets
P = ["p"+str(i+1) for i in range(p)] # set of PDP
N = ["n"+str(i+1) for i in range(n)] # set of altruistic bridge donors
V = P
c = 2
L = [l+1 for l in range(c)]



# Objective Function 

In [3]:
# Create the LP problem
prob = pulp.LpProblem('Kidney_Exchange', LpMaximize)

In [4]:
vars = LpVariable.dicts("x", (V,P,L), lowBound=0,upBound=1,cat="Integer")

for v in V:
    for p in P:
        for l in L:
            if v == p: del(vars[v][p][l])
vars

{'p1': {'p1': {},
  'p2': {1: x_p1_p2_1, 2: x_p1_p2_2},
  'p3': {1: x_p1_p3_1, 2: x_p1_p3_2},
  'p4': {1: x_p1_p4_1, 2: x_p1_p4_2}},
 'p2': {'p1': {1: x_p2_p1_1, 2: x_p2_p1_2},
  'p2': {},
  'p3': {1: x_p2_p3_1, 2: x_p2_p3_2},
  'p4': {1: x_p2_p4_1, 2: x_p2_p4_2}},
 'p3': {'p1': {1: x_p3_p1_1, 2: x_p3_p1_2},
  'p2': {1: x_p3_p2_1, 2: x_p3_p2_2},
  'p3': {},
  'p4': {1: x_p3_p4_1, 2: x_p3_p4_2}},
 'p4': {'p1': {1: x_p4_p1_1, 2: x_p4_p1_2},
  'p2': {1: x_p4_p2_1, 2: x_p4_p2_2},
  'p3': {1: x_p4_p3_1, 2: x_p4_p3_2},
  'p4': {}}}

In [5]:
{v.name: v.varValue for v in prob.variables()}

{}

# LP problem

In [6]:
# add weight coefficient
np.random.seed(3424)
weight = list((vars[v][p][l],np.random.uniform(0,1)) for  v in V for p in P for l in L if v != p)

In [7]:
count = 1

# objective function
#1
prob += LpAffineExpression(weight)

# constraints
#2
for v in V:
    for l in L: 
        prob += LpConstraint(lpSum(vars[p][v][l] for p in P if v != p) - lpSum(vars[v][p][l] for p in P if v!= p),sense = LpConstraintLE, rhs = 0, name = count)
        count+= 1
#3
for v in V: 
    prob += LpConstraint(lpSum(vars[v]), sense = LpConstraintLE, rhs = 1, name = count)
    count += 1
#4
for l in L:
    prob += LpConstraint(lpSum([vars[v][p][l] for v in  V for p in P if v!=p]), sense = LpConstraintLE, rhs = K, name = count)
    count += 1

#5
for l in L:
    for v in V:
        if int(v[1:]) < l:
            prob += LpConstraint(lpSum(vars[v][p][l] for p in P if v!= p) - lpSum(vars['p'+str(l)][p][l] for p in P if 'p'+str(l) !=p), sense = LpConstraintLE, rhs =  0, name = count)
            count += 1

#6
for l in L:
    for v in V:
        if int(v[1:]) > l:
            prob += LpConstraint(lpSum(vars[v][p][l] for p in P if v!= p), sense = LpConstraintLE, rhs = 0, name = count)
            count += 1

In [8]:
print(prob)

Kidney_Exchange:
MAXIMIZE
0.7423662030329226*x_p1_p2_1 + 0.9956432998821247*x_p1_p2_2 + 0.3201162031897976*x_p1_p3_1 + 0.944754925041174*x_p1_p3_2 + 0.8128383923588507*x_p1_p4_1 + 0.7602489260663715*x_p1_p4_2 + 0.9504909986432636*x_p2_p1_1 + 0.9062308528503263*x_p2_p1_2 + 0.39695548414762005*x_p2_p3_1 + 0.007338214285963152*x_p2_p3_2 + 0.6390014361899816*x_p2_p4_1 + 0.013731044118606595*x_p2_p4_2 + 0.04317062465523169*x_p3_p1_1 + 0.5626229061541081*x_p3_p1_2 + 0.0190587008116333*x_p3_p2_1 + 0.5691750402139145*x_p3_p2_2 + 0.4404742293683862*x_p3_p4_1 + 0.2984115955619401*x_p3_p4_2 + 0.05758136192174834*x_p4_p1_1 + 0.9488090662523656*x_p4_p1_2 + 0.5761509684076184*x_p4_p2_1 + 0.07878365617147842*x_p4_p2_2 + 0.7366116182796062*x_p4_p3_1 + 0.1761458652970901*x_p4_p3_2 + 0
SUBJECT TO
1: - x_p1_p2_1 - x_p1_p3_1 - x_p1_p4_1 + x_p2_p1_1 + x_p3_p1_1 + x_p4_p1_1
 <= 0

2: - x_p1_p2_2 - x_p1_p3_2 - x_p1_p4_2 + x_p2_p1_2 + x_p3_p1_2 + x_p4_p1_2
 <= 0

3: x_p1_p2_1 - x_p2_p1_1 - x_p2_p3_1 - x_p2_p4

# Solve the Problem

In [9]:
prob.solve()

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /opt/anaconda3/lib/python3.7/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/mn/d2qgjlw519bgr2j1828v0s2m0000gn/T/241169f00d9546db8380850ae0d41031-pulp.mps max timeMode elapsed branch printingOptions all solution /var/folders/mn/d2qgjlw519bgr2j1828v0s2m0000gn/T/241169f00d9546db8380850ae0d41031-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 25 COLUMNS
At line 215 RHS
At line 236 BOUNDS
At line 261 ENDATA
Problem MODEL has 20 rows, 24 columns and 117 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 1.90187 - 0.00 seconds
Cgl0002I 15 variables fixed
Cgl0008I 4 inequality constraints converted to equality constraints
Cgl0003I 0 fixed, 0 tightened bounds, 0 strengthened rows, 2 substitutions
Cgl0004I processed model has 0 rows, 0 columns (0 integer (0 of which binary)) and 0 elem

1

In [10]:
for a in prob.variables():
    if a.varValue == 1:
        print(a.name, "=", a.varValue)

x_p1_p2_2 = 1.0
x_p2_p1_2 = 1.0


# Matrix and Determinant

In [11]:
# create empty dataframe
column_names = ["x_" + v + "_" + p + "_" + str(c+1) for c in range(c) for v in V for p in P if p != v]
coef = pd.DataFrame(columns = column_names)

# add coefficient for each row
for i in range(1,count):
    c = prob.constraints[str(i)].toDict()['coefficients']
    c_df = pd.DataFrame(c)
    c_df = c_df.set_index('name')
    coef = coef.append(c_df.T, ignore_index = True)

# fill empty value with 0
coef.fillna(0, inplace = True)

In [12]:
coef

,x_p1_p2_1,x_p1_p3_1,x_p1_p4_1,x_p2_p1_1,x_p2_p3_1,x_p2_p4_1,x_p3_p1_1,x_p3_p2_1,x_p3_p4_1,x_p4_p1_1,...,x_p1_p4_2,x_p2_p1_2,x_p2_p3_2,x_p2_p4_2,x_p3_p1_2,x_p3_p2_2,x_p3_p4_2,x_p4_p1_2,x_p4_p2_2,x_p4_p3_2
0,-1,-1,-1,1,0,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,-1,1,0,0,1,0,0,1,0,0
2,1,0,0,-1,-1,-1,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,-1,-1,-1,0,1,0,0,1,0
4,0,1,0,0,1,0,-1,-1,-1,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,-1,-1,-1,0,0,1
6,0,0,1,0,0,1,0,0,1,-1,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,1,0,0,1,0,0,1,-1,-1,-1
8,1,1,1,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
9,0,0,0,1,1,1,0,0,0,0,...,0,1,1,1,0,0,0,0,0,0


In [13]:
comb = list(itertools.combinations(V,2))
del_list = list()
for (a,b) in comb:
    for l in L:
        variable = 'x_' + a + '_' + b + "_" + str(l)
        del_list.append(variable)

In [14]:
del_list

['x_p1_p2_1',
 'x_p1_p2_2',
 'x_p1_p3_1',
 'x_p1_p3_2',
 'x_p1_p4_1',
 'x_p1_p4_2',
 'x_p2_p3_1',
 'x_p2_p3_2',
 'x_p2_p4_1',
 'x_p2_p4_2',
 'x_p3_p4_1',
 'x_p3_p4_2']

In [15]:
coef = coef.drop(del_list,axis =1)

In [16]:
A = np.matrix(coef)

In [17]:
rank = matrix_rank(A)

In [18]:
rank

10

In [19]:
var_list = list(coef)
var_comb = list(itertools.combinations(var_list, rank))

In [20]:
cons_list = list(range(count-1))
cons_comb = list(itertools.combinations(cons_list, rank))

In [21]:
for c in cons_comb:
    sub = coef.iloc[list(c)]
    if matrix_rank(sub) < rank:
        cons_comb.remove(c)

In [ ]:
max_det = 0
list_det = list()
for c in cons_comb:
    for v in var_comb:
        sub = coef.iloc[list(c)]
        sub = sub[list(v)]
        if matrix_rank(sub) == rank:
            det = int(abs(np.linalg.det(sub)))
            if det > max_det:
                max_det = det
            if not((det & (det-1) == 0)):
                list_det.append(sub)

In [ ]:
max_det

In [ ]:
list_det

In [ ]:
#start_time = time.time()
#for c in cons_comb:
#    for v in var_comb:
#        sub = coef.iloc[list(c)]
#        sub = sub[list(v)]
#        det = int(abs(np.linalg.det(sub)))
#        if det > max_det:
#            max_det = det
#        if not((det & (det-1) == 0)):
#          list_det.append(sub)
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#from scipy.sparse.linalg import splu


# compute determinant using first row
#def det(M):
#    l = sp.sparse.tril(sub)
#    u = sp.sparse.triu(sub)
#    diagL = diagL.astype(np.complex128)
#    diagU = diagU.astype(np.complex128)
#    logdet = np.log(diagL).sum() + np.log(diagU).sum()
#    return np.exp(logdet)

In [ ]:
#max_det = 0
#list_det = list()
#start_time = time.time()
#for c in cons_comb:
#    for v in var_comb:
#        sub = coef.iloc[list(c)]
#        sub = sub[list(v)]
#        det = int(abs(linalg.det(sub)))
#        if det > max_det:
#            max_det = det
#        if not((det & (det-1) == 0)):
#          list_det.append(sub)
#print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
#np.diag((sp.sparse.tril(sub)))


In [ ]:
#max_det
#list_det

In [ ]:
#sub.to_numpy()